**Problem Statement**: We will be designing the Question Answer System on the twitter dataset that is extracted for the Uttarakhand flash lood related topic.    

**Approach**: We will be using teh cdqa(Closed Domain question answering system) package which is built on top of Hugging face transformers library.


In [1]:
## Installing the spacy for preprocessing and data cleaning.
!pip install -U spacy
!pip install -U spacy-lookups-data
# python -m spacy download en_core_web_sm
# python -m spacy download en_core_web_md
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 12.8MB 244kB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 460kB 50.9MB/s 
     |████████████████████████████████| 9.1MB 47.3MB/s 
     |████████████████████████████████| 1.1MB 50.1MB/s 
     |████████████████████████████████| 122kB 55.2MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=dcf579f358235bebd7011af94f6c150c3a961bc2d3e454016cc5b61675f2e9a4
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.2.0 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 0.25.0 which is incompatible.
ERROR: cdqa 1.3.9 has requirement tqdm==4.32.2, but you'll have tqdm 4.59.0 which is incompatible.
  Found existing installation: smart-open 4.2.0
    Uninst

In [2]:
## Installing the cdqa package 
!pip install cdqa

  Using cached https://files.pythonhosted.org/packages/9f/3d/7a6b68b631d2ab54975f3a4863f3c4e9b26445353264ef01f465dc9b0208/tqdm-4.32.2-py2.py3-none-any.whl
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.2.0 which is incompatible.
ERROR: spacy 3.0.5 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.32.2 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 0.25.0 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.32.2 which is incompatible.
  Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0


In [3]:
## Importing all the required libraries
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import os
import pandas as pd
from ast import literal_eval
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
## Creating a dataframe which has all the tweets from the csv file.
header_list = ["paragraph"]
df = pd.read_csv('/content/drive/MyDrive/Internship_Assignments/ROOT2AI/Uttarakhand.csv', names=header_list, converters={'paragraphs': literal_eval})
df = df[1:]
df.head()

,paragraph
1,Horrible news out of #Uttarakhand
2,Prayer for #Uttarakhand 🙏🙏
3,Ohhh God Please Save #Uttarakhand
4,Extremely Sad To Hear About The #Uttarakhand G...
5,Prayers for #Uttarakhand 🙏🙏


Now we will do the data cleaning task. This involves the following:

1. Removing un necessary emails.
2. Removing un necessary links.
3. Removing all the special characters like: signs and emojis etc.

In [5]:
## Removing emails
import re
df['paragraph'] = df['paragraph'].apply(lambda x: re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', ' ', x))

In [6]:
pd.set_option('display.max_colwidth',-1)
df

,paragraph
1,Horrible news out of #Uttarakhand
2,Prayer for #Uttarakhand 🙏🙏
3,Ohhh God Please Save #Uttarakhand
4,"Extremely Sad To Hear About The #Uttarakhand Glacier Break, Praying For The Safety Of Missing People... 🙏\n\nHelpline… https://t.co/a8HXQ6Y3Hb"
5,Prayers for #Uttarakhand 🙏🙏
6,Prayer for #Uttarakhand.\n🙏🙏
7,ALERT 🚨 #Uttarakhand \n\nEmergency 🆘 numbers 1070 9557444486 \n\nPlease follow all official advisories.\n\n#Uttarakhand #Chamoli
8,"Heavy Devastation Due To Glacier breakage in Chamoli, Uttrakhand.Very sad news...Geographical issues of our country… https://t.co/hFmkb6D9bX"
9,I wish everyone stay safe in Uttarakhand during this tragedy.\n\n#Uttarakhand
10,O Lord Shiva please save #Uttarakhand !


In [7]:
## removing all the un necessary links 
df['paragraph'] = df['paragraph'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))

In [8]:
df

,paragraph
1,Horrible news out of #Uttarakhand
2,Prayer for #Uttarakhand 🙏🙏
3,Ohhh God Please Save #Uttarakhand
4,"Extremely Sad To Hear About The #Uttarakhand Glacier Break, Praying For The Safety Of Missing People... 🙏\n\nHelpline…"
5,Prayers for #Uttarakhand 🙏🙏
6,Prayer for #Uttarakhand.\n🙏🙏
7,ALERT 🚨 #Uttarakhand \n\nEmergency 🆘 numbers 1070 9557444486 \n\nPlease follow all official advisories.\n\n#Uttarakhand #Chamoli
8,"Heavy Devastation Due To Glacier breakage in Chamoli, Uttrakhand.Very sad news...Geographical issues of our country…"
9,I wish everyone stay safe in Uttarakhand during this tragedy.\n\n#Uttarakhand
10,O Lord Shiva please save #Uttarakhand !


In [9]:
## Removing all the un necessary signs and emojis
df['paragraph'] = df['paragraph'].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+', '', x))

In [10]:
df

,paragraph
1,Horrible news out of Uttarakhand
2,Prayer for Uttarakhand
3,Ohhh God Please Save Uttarakhand
4,Extremely Sad To Hear About The Uttarakhand Glacier Break Praying For The Safety Of Missing People Helpline
5,Prayers for Uttarakhand
6,Prayer for Uttarakhand
7,ALERT Uttarakhand Emergency numbers 1070 9557444486 Please follow all official advisoriesUttarakhand Chamoli
8,Heavy Devastation Due To Glacier breakage in Chamoli UttrakhandVery sad newsGeographical issues of our country
9,I wish everyone stay safe in Uttarakhand during this tragedyUttarakhand
10,O Lord Shiva please save Uttarakhand


Now we have to convert the dataframe to a structure that we can fit it to the cdqa pipeline. It should have a title column and a paragraphs column.

In [11]:
## Creating the paragraphs columns which should be in the list format
st=[]

for i in range(df.shape[0]):
  va =[]
  va.append(df['paragraph'].iloc[i])
  st.append(va)

df['paragraphs'] = st

In [12]:
## Creating a title column
title = []
for i in range(df.shape[0]):
  title.append("tweet"+str(i))
df['title']=title

In [13]:
## Creating the final dataframe which had only two columns - title and paragraphs
df = df[['title','paragraphs']]

In [14]:
df

,title,paragraphs
1,tweet0,[Horrible news out of Uttarakhand]
2,tweet1,[Prayer for Uttarakhand ]
3,tweet2,[Ohhh God Please Save Uttarakhand]
4,tweet3,[Extremely Sad To Hear About The Uttarakhand Glacier Break Praying For The Safety Of Missing People Helpline ]
5,tweet4,[Prayers for Uttarakhand ]
6,tweet5,[Prayer for Uttarakhand]
7,tweet6,[ALERT Uttarakhand Emergency numbers 1070 9557444486 Please follow all official advisoriesUttarakhand Chamoli]
8,tweet7,[Heavy Devastation Due To Glacier breakage in Chamoli UttrakhandVery sad newsGeographical issues of our country ]
9,tweet8,[I wish everyone stay safe in Uttarakhand during this tragedyUttarakhand]
10,tweet9,[O Lord Shiva please save Uttarakhand ]


In [15]:
## Downloading the pretrained model trained on the squad dataset
download_model('bert-squad_1.1', dir='./models')


bert_qa.joblib already downloaded


In [16]:
## Fitting the model on the dataframe
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib',max_df=1.0)
cdqa_pipeline.fit_retriever(df=df)

QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

Now we can start asking question to the model and it will predict the first 3 probable answer with the confidence level and also the tweete in which the answer appears(as we have prepared that in our data preprocessing).

We will be asking the following question as extracted from the tweets:

1. How many people feared to be drowned in flood after glacier bursts?
2. What is the water level at Rishikesh?
3. What is the helpline number?
4. How many personnel deployed on rescue mission?
5. How many casualties feared in the flash flood?
6. Who is Chief Secretary of Uttrakhand?
7. glacier burst in which village?
8. Why there is floods in Uttrakhand?
9. Flood in which river?
10. Who is the prime minister?
11. Who is the home minister?
12. Which helicopter is on standby for search?
13. What is the helpline number of Uttrakhand government?

In [17]:
query = "How many people feared to be drowned in flood after glacier bursts?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('150',
  'tweet288',
  '150 people feared to be drowned in flood after glacier bursts UttarakhandUttarakhandDisaster ',
  24.431776278057452),
 ('100-150',
  'tweet2314',
  'UPDATE 100-150 feared dead as glacier bursts in India Uttarakhand',
  15.706282849207597),
 ('150',
  'tweet1342',
  'Uttarakhand flood Glacier bursts in Chamoli district 150 people missingflood ',
  14.630708087777222)]

In [18]:
query = "What is the water level at Rishikesh?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('34050 m',
  'tweet62',
  'Water level at Rishikesh the water Level may rise to 34050 m at 2000 hrs and Haridwar the Water Level May Reach 2 ',
  14.079714504650566),
 ('34050 m',
  'tweet236',
  'Water level at Rishikesh the water Level may rise to 34050 m at 2000 hrs and Haridwar the Water Level May Reach 2 ',
  14.079712335041496),
 ('34050 m',
  'tweet280',
  'Water level at Rishikesh the water Level may rise to 34050 m at 2000 hrs and Haridwar the Water Level May Reach 2 ',
  14.079712335041496)]

In [19]:
query = "What is the helpline number?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('9557444486',
  'tweet1888',
  'Helpline number forUttarakhand 9557444486 ',
  15.463002726079441),
 ('1070',
  'tweet2783',
  'Uttarakhand Helpline number-1070 ',
  15.372154531288299),
 ('1070',
  'tweet5874',
  'Uttarakhand CM tsrawatbjp shares the helpline number 1070 or 9557444486 ',
  12.8993750835637)]

In [20]:
query = "How many personnel deployed on rescue mission?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('250',
  'tweet451',
  '3 teams of ITBPofficial comprising of 250 personnel deployed on rescue mission in Uttarakhand AsianetNewsEN',
  21.025092420241524),
 ('Hundreds',
  'tweet4780',
  'Hundreds of ITBP personnel has been deployed for rescue of casualties in flood affected areas of Chamoli district ',
  15.922582882277663),
 ('200',
  'tweet4959',
  '4 districts of Uttarakhand on alert 200 ITBP personnel deployed for rescue operation Joshimath Chamoli ',
  14.913025039069348)]

In [21]:
query = "How many casualties feared in the flash flood?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('100-150',
  'tweet4522',
  '100-150 casualties feared in the flash flood in Chamoli districtUttarakhand',
  20.558219041844403),
 ('100-150',
  'tweet2870',
  '100-150 casualties feared in the flash flood  Uttarakhand Chief Secretary',
  19.91663281120205),
 ('100-150',
  'tweet1534',
  '100-150 casualties feared in flash flood in Chamoli Ukhand Chief Secretary  via ',
  19.484737412108423)]

In [22]:
query = "Who is Chief Secretary of Uttrakhand?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('OM Prakash',
  'tweet4538',
  '100-150 casualties feared in the flash flood in Chamoli district Uttarakhand Chief Secretary OM Prakash to ANITV9News',
  15.040079275274248),
 ('OM Prakash',
  'tweet7124',
  '100-150 casualties feared in the flash flood in Chamoli district Uttarakhand Chief Secretary OM Prakash',
  14.991816631460164),
 ('OM Prakash',
  'tweet4355',
  '100-150 casualties feared in the flash flood in Chamoli district Uttarakhand Chief Secretary OM Prakash ',
  14.991813532018636)]

In [23]:
query = "glacier burst in which village?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('Reni',
  'tweet7198',
  'Very disturbing newsA glacier burst at the Reni village of Chamoli district in Uttarakhand has triggered a massi ',
  14.34512272702122),
 ('Reni',
  'tweet7227',
  'A glacier burst at the Reni village of Chamoli district in Uttarakhand has triggered a massive flood in the ',
  13.81416020314221),
 ('Alaknanda valley',
  'tweet6035',
  'Amateur video This mornings episode Glacier burst in Alaknanda valley Please verify naturaldisaster glacier ',
  13.497976366003162)]

In [24]:
query = "Why there is floods in Uttrakhand?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('glacier burst',
  'tweet2391',
  'Massive floods in Uttarakhand due to glacier burst Pray God that the lives be saved Chamoli Uttrakhand',
  8.261201558758607),
 ('melting of a Glacier',
  'tweet548',
  'Flash Floods in Uttarakhand due to melting of a GlacierUnprecedented floods in Hyderabad last  yearAll are sig ',
  5.921102253851397),
 ('Stay safe',
  'tweet2925',
  'Stay safe Uttrakhand pplUttarakhand ',
  -0.7746857262299653)]

In [25]:
query = "Flood in which river?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('dhauliganga river',
  'tweet1167',
  'A massive flood was triggered in the dhauliganga river in Uttarakhand Chamoli district after a glacier broke of ',
  14.33580113344141),
 ('Dhauliganga River',
  'tweet6018',
  'Scary videos coming in from Uttarakhand Massive flood in the Dhauliganga River Uttarakhand Flood ',
  13.981613870058375),
 ('Dhauliganga river',
  'tweet5029',
  'A massive flood in the Dhauliganga river was reported near a village in Uttarakhands Chamoli district ',
  13.940731490711817)]

In [26]:
query = "Who is the prime minister?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('NarendraModi',
  'tweet1322',
  'Prime Minister NarendraModi and Union Home Minister AmitShah on Sunday spoke to Uttarakhand Chief Minister ',
  18.213412711613465),
 ('Shri narendramodi',
  'tweet1675',
  'Prime Minister Shri narendramodi speaks to Uttarakhand CM Shri tsrawatbjp  reviews unfortunate situation in ',
  15.068045386417204),
 ('Narendra Modi',
  'tweet2105',
  'Prime Minister Narendra Modi said I am constantly monitoring unfortunate situation in Uttarakhand India stands w ',
  14.281278664963347)]

In [27]:
query = "Who is the home minister?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('AmitShah',
  'tweet4335',
  'BreakingNews Home Minister AmitShah has spoken to Uttarakhand Chief Minister TrivendraSinghRawat as well as ITB ',
  16.462706038222862),
 ('Amit Shah',
  'tweet7649',
  'WATCH Uttarakhand glacier burst Union Home Minister Amit Shah assures assistance ',
  16.388332623663384),
 ('AmitShah',
  'tweet1499',
  'Home Minister AmitShah assures Uttarakhand CM tsrawatbjp of all possible support to deal with the situation aris ',
  15.997527212987878)]

In [28]:
query = "Which helicopter is on standby for search?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('Chinook',
  'tweet1184',
  'WeStandTogetherForUttrakhandIAF Mi-17 amp Chinook helicopters from Chandigarh and elsewhere on standby for search amp ',
  12.447783352696682),
 ('amp Chinook helicopters',
  'tweet1912',
  'IAF Mi-17 amp Chinook helicopters from Chandigarh and elsewhere on standby for search amp rescue ops in Uttarakhand W ',
  10.962969078141896),
 ('amp Chinook helicopters',
  'tweet7572',
  'IAF Mi-17 amp Chinook helicopters from Chandigarh and elsewhere on standby for search amp rescue ops in Uttarakhand W ',
  10.962969078141896)]

In [29]:
query = "What is the helpline number of Uttrakhand government?"
prediction = cdqa_pipeline.predict(query,3)
prediction

[('25000 Thousand',
  'tweet579',
  'Uttarakhand I Am Happy To Announce That I Help Uttrakhand Government To Give 25000 Thousand To Uttrakhand Governm ',
  16.826341339535468),
 ('9557444486',
  'tweet1888',
  'Helpline number forUttarakhand 9557444486 ',
  15.657681437970613),
 ('1070',
  'tweet410',
  'Natural disaster in UttarakhandLet all our pray for Uttrakhand Government has issued helpline numbers - 1070 ',
  15.30251492643737)]